# Model Server

In [11]:
# nuclio: ignore
import nuclio

In [12]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel
%nuclio config spec.image = "mlrun/mlrun"
%nuclio config kind = "job"

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable
%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'
%nuclio: setting kind to 'job'


In [13]:
%%nuclio cmd -c
python -m pip install numpy cloudpickle v3io sklearn

In [14]:
from cloudpickle import load
from typing import List
from datetime import datetime
from sklearn.datasets import load_iris

import os
import numpy as np
import mlrun

class ClassifierModel(mlrun.runtimes.MLModelServer):
    def load(self):
        """Load model from storage."""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample.
        
        :param body : A dict of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.model.predict(feats)
            resp = result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")
        
        return resp

In [15]:
# nuclio: end-code

# Test models locally and deploy

The sklearn-project generated one or more models that will be deployed in the server project `sklearn-servers`

### Test locally

In [16]:
models_path = ['https://s3.wasabisys.com/iguazio/models/iris/model.pkl']

iris = load_iris()

x = iris['data'].tolist()
y = iris['target']

for model in models_path:
    if model.endswith(".pkl"):
        
        my_server = ClassifierModel('classifier', model_dir=model)
        my_server.load()

        a = my_server.predict({"instances": x})
        
        assert len(a)==150

## Document and save

In [17]:
fn = mlrun.new_model_server('model-server', model_class='ClassifierModel')
fn.spec.description = "generic sklearn model server"
fn.metadata.categories = ['serving', 'ml']
fn.metadata.labels = {'author': 'yaronh', 'framework': 'sklearn'}
fn.export()

> 2020-10-18 17:27:05,549 [info] function spec saved to path: function.yaml


##  Deploy server

In [18]:
user_name = os.getenv('V3IO_USERNAME')

fn.apply(mlrun.mount_v3io())
fn.set_envs({'SERVING_MODEL_iris_dataset_v1': models_path[0],
             'INFERENCE_STREAM': 'users/{}/tststream'.format(user_name)})

address = fn.deploy(project='sk-project')

> 2020-10-18 17:27:05,558 [info] deploy started
[nuclio] 2020-10-18 17:27:14,770 (info) Build complete
[nuclio] 2020-10-18 17:27:33,009 (info) Function deploy complete
[nuclio] 2020-10-18 17:27:33,017 done updating sk-project-model-server, function address: 192.168.224.209:30499
> 2020-10-18 17:27:33,026 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match


##  Test server

In [19]:
predict_url = address+"/iris_dataset_v1/predict"
my_data = '''{"instances":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''
!curl {predict_url} -d '{my_data}'

[0, 2]